In [1]:
import pandas as pd

In [2]:
xls = pd.ExcelFile(r"data/‏‏raw.xlsx")

### Transform sheets to column

#### columns

In [3]:
aliases =  {
    'raw_id': [';', 'קוד', 'קוד מטופל', 'קוד נבדק'],
    'clinic_application_date': ['תאריך פנייה למרפאה'],
    'intake_date': ['תאריך אינטייק'],
    'group': ['הקצאה רנדומלית', 'סוג טיפול'],
    'signing_date': ['תאריך חתימה'],
    'therapy_start_date': ['תאריך תחילת התערבות'],
    'therapy_end_date': ['שאלוני סוף התערבות (8 שבועות)', 'תאריך סיום התערבות'],
    'suiteable_for_pp' ['מתאים לתומך ע"י קלינאי 1. כן 0. לא', 'מתאים לתומך ע"י קלינאי 1. כן 0. לא', 'מתאימים לתומך? 1.כן 0.לא', 'מתאימים לתומך? 1.כן 0.לא']
}

rename_map = {col: key for key, value in aliases.items() for col in value}
columns = list(aliases.keys()) + ['sheet', 'clean_id']

In [4]:
def drop_s(x):
    if pd.isna(x):
        return x
    elif x.endswith('s'):
        return x[:-1]
    else: 
        return x


In [5]:
data_sets = []

for sheet in xls.sheet_names:
    sheet_df = xls.parse(sheet)
    sheet_df['sheet'] = sheet
    sheet_df.rename(columns=rename_map, inplace=True, errors='ignore')
    
    sheet_df["clean_id"] = sheet_df["raw_id"].astype(str).str.lower()
    sheet_df["clean_id"] = sheet_df["clean_id"].apply(drop_s)

    sheet_columns = [col for col in columns if col in sheet_df.columns]
    sheet_df = sheet_df[sheet_columns]
    data_sets.append(sheet_df)


df = pd.concat(data_sets)

### ID processing

In [6]:
df.to_csv("data\preprocessed.csv", index=0)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\USER\AppData\Local\Temp\ipykernel_4308\4067398792.py:1: SyntaxWarning: invalid escape sequence '\p'
  df.to_csv("data\preprocessed.csv", index=0)
